In [3]:
import sys
sys.path.insert(1, '../Src')

import pandas as pd
import os
from utils.preprocessing import *

PATH_DATASET = '../Dataset/data-h-m1-challenge-final'
PATH_RESULTS = '../Results/dataset/'

In [4]:
df_sales = pd.read_csv(os.path.join(PATH_DATASET,'sales.csv'))
df_sales = columns_to_int(df_sales,['S100','I100','C100','C101'])
df_sales = get_item_id(df_sales)
df_sales.head(2)

,DATE,S100,I100,C100,C101,QTT,item_id
0,2017-01-08,0,0,12,76,2,0_12_76
1,2017-01-08,0,0,12,149,3,0_12_149


In [5]:
df_submission_sample = pd.read_csv(os.path.join(PATH_DATASET,'submission_sample.csv'))
df_submission_sample[['DATE', 'S100', 'I100', 'C100', 'C101']] = df_submission_sample['ID'].str.split('_',expand=True)
df_submission_sample = columns_to_int(df_submission_sample,['S100','I100','C100','C101'])
df_submission_sample = get_item_id(df_submission_sample)
df_submission_sample.head(2)

,ID,QTT,DATE,S100,I100,C100,C101,item_id
0,2021-12-19_0_1_12_76,-1,2021-12-19,0,1,12,76,1_12_76
1,2021-12-19_0_1_12_140,-1,2021-12-19,0,1,12,140,1_12_140


In [6]:
df_sales['DATE'] = pd.to_datetime(df_sales['DATE'], errors='coerce')
df_submission_sample['DATE'] = pd.to_datetime(df_submission_sample['DATE'], errors='coerce')

date = df_sales['DATE'].min()
maxi = df_submission_sample['DATE'].max()

dict_dates = {}
idx = 0
while(True):    
    if date>maxi:
        break
    dict_dates[date]=idx
    date = date+ pd.DateOffset(days=7)    
    idx = idx+1

df_sales['date_block_num'] = df_sales['DATE'].replace(dict_dates)
df_submission_sample['date_block_num'] = df_submission_sample['DATE'].replace(dict_dates)

In [7]:
df_store = pd.read_csv(os.path.join(PATH_DATASET,'meta-store.csv'))
df_store = columns_to_int(df_store,['S100','S101','S102','S103'])
df_store.head(2)

,S100,S101,S102,S103
0,0,1,17,10
1,1,1,18,10


In [8]:
df_item = pd.read_csv(os.path.join(PATH_DATASET,'meta-item.csv'))
df_item = columns_to_int(df_item,['I100','I101','I102','I103'])
df_item.head(2)

,I100,I101,I102,I103
0,0,2,1,1
1,1,2,1,1


In [9]:
for column_name in ['S100','I100','C100','C101','item_id']:
    ids_sales,ids_submission = unique_comparations(df_sales,df_submission_sample,column_name)

unique S100 :  31
unknown S100 in submission : set()
unique I100 :  96
unknown I100 in submission : set()
unique C100 :  21
unknown C100 in submission : set()
unique C101 :  355
unknown C101 in submission : set()
unique item_id :  3781
unknown item_id in submission : set()


In [10]:
df_submission_sample[(df_submission_sample['I100']=='15') & (df_submission_sample['C100']=='14')].head(2)

,ID,QTT,DATE,S100,I100,C100,C101,item_id,date_block_num


In [11]:
df_sales[(df_sales['I100']=='15') & (df_sales['C101']=='192')]

,DATE,S100,I100,C100,C101,QTT,item_id,date_block_num


In [12]:
df_sales[(df_sales['C101']=='192')]

,DATE,S100,I100,C100,C101,QTT,item_id,date_block_num


In [13]:
df_sales[(df_sales['I100']=='15') & (df_sales['C100']=='12')].head(2)

,DATE,S100,I100,C100,C101,QTT,item_id,date_block_num


In [14]:
%timeit df_sales.to_pickle(os.path.join(PATH_RESULTS,'sales.pkl'))

953 ms ± 7.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit df_submission_sample.to_pickle(os.path.join(PATH_RESULTS,'submission_sample.pkl'))

29.2 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit df_item.to_pickle(os.path.join(PATH_RESULTS,'item.pkl'))

3.54 ms ± 542 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit df_store.to_pickle(os.path.join(PATH_RESULTS,'store.pkl'))

3.5 ms ± 574 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
